In [ ]:
# Comodo import
from comodo.drakeSimulator.drakeSimulator import DrakeSimulator
from comodo.mujocoSimulator.mujocoSimulator import MujocoSimulator
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.centroidalMPC.centroidalMPC import CentroidalMPC
from comodo.centroidalMPC.mpcParameterTuning import MPCParameterTuning
from comodo.TSIDController.TSIDParameterTuning import TSIDParameterTuning
from comodo.TSIDController.TSIDController import TSIDController

# optimisation import
import pygmo as pg

# General  import 
import xml.etree.ElementTree as ET
import numpy as np
import tempfile
import urllib.request
import logging
import time
import os
from contextlib import redirect_stderr, redirect_stdout

In [ ]:
# set the logging level
logging.getLogger().setLevel(logging.INFO)

In [ ]:
# Getting stickbot urdf file and convert it to string 
urdf_robot_file = tempfile.NamedTemporaryFile(mode="w+")
url = 'https://raw.githubusercontent.com/icub-tech-iit/ergocub-gazebo-simulations/master/models/stickBot/model.urdf'
urllib.request.urlretrieve(url, urdf_robot_file.name)
# Load the URDF file
tree = ET.parse(urdf_robot_file.name)
root = tree.getroot()

# Convert the XML tree to a string
robot_urdf_string_original = ET.tostring(root)

create_urdf_instance = createUrdf(
    original_urdf_path=urdf_robot_file.name, save_gazebo_plugin=False
)

In [ ]:
# Define parametric links and controlled joints  
legs_link_names = ["hip_3", "lower_leg"]
joint_name_list = [
    "r_shoulder_pitch",
    "r_shoulder_roll",
    "r_shoulder_yaw",
    "r_elbow",
    "l_shoulder_pitch",
    "l_shoulder_roll",
    "l_shoulder_yaw",
    "l_elbow",
    "r_hip_pitch",
    "r_hip_roll",
    "r_hip_yaw",
    "r_knee",
    "r_ankle_pitch",
    "r_ankle_roll",
    "l_hip_pitch",
    "l_hip_roll",
    "l_hip_yaw",
    "l_knee",
    "l_ankle_pitch",
    "l_ankle_roll",
]

In [ ]:
# Define the robot modifications
modifications = {}
for item in legs_link_names:
    left_leg_item = "l_" + item
    right_leg_item = "r_" + item
    modifications.update({left_leg_item: 1.2})
    modifications.update({right_leg_item: 1.2})
# Motors Parameters 
Im_arms = 1e-3*np.ones(4) # from 0-4
Im_legs = 1e-3*np.ones(6) # from 5-10
kv_arms = 0.001*np.ones(4) # from 11-14
kv_legs = 0.001*np.ones(6) #from 20

Im = np.concatenate((Im_arms, Im_arms, Im_legs, Im_legs))
kv = np.concatenate((kv_arms, kv_arms, kv_legs, kv_legs))

In [ ]:
# Modify the robot model and initialize
create_urdf_instance.modify_lengths(modifications)
urdf_robot_string = create_urdf_instance.write_urdf_to_file()
create_urdf_instance.reset_modifications()

In [35]:
def cost_function(x_k, simulator="drake", urdf_robot_string=urdf_robot_string, visualise_flag = False):
    with redirect_stderr(open(os.devnull, "w")):
        robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)
        s_des, xyz_rpy, H_b = robot_model_init.compute_desired_position_walking()
    # Define simulator and initial position
    if simulator == "drake":
        sim_instance = DrakeSimulator()
        sim_instance.set_visualize_robot_flag(visualise_flag)
        sim_instance.load_model(robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im)

    elif simulator == "mujoco":
        sim_instance = MujocoSimulator()
        sim_instance.load_model(robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im)
        sim_instance.set_visualize_robot_flag(visualise_flag)
    
    # sim_instance.load_model(robot_model_init, s=s_des, xyz_rpy=xyz_rpy, kv_motors=kv, Im=Im)
    s, ds, tau = sim_instance.get_state()
    t = sim_instance.get_simulation_time()
    H_b = sim_instance.get_base()
    w_b = sim_instance.get_base_velocity()
    # Controller Parameters
    tsid_parameter = TSIDParameterTuning()
    mpc_parameters = MPCParameterTuning()
    if x_k is not None:
        tsid_parameter.set_from_x_k(x_k)

    # TSID Instance
    TSID_controller_instance = TSIDController(frequency=0.01, robot_model=robot_model_init)
    TSID_controller_instance.define_tasks(tsid_parameter)
    TSID_controller_instance.set_state_with_base(s, ds, H_b, w_b, t)

    # MPC Instance
    step_lenght = 0.1
    mpc = CentroidalMPC(robot_model=robot_model_init, step_length=step_lenght)
    mpc.intialize_mpc(mpc_parameters=mpc_parameters)

    # Set desired quantities
    mpc.configure(s_init=s_des, H_b_init=H_b)
    TSID_controller_instance.compute_com_position()
    mpc.define_test_com_traj(TSID_controller_instance.COM.toNumPy())

    # TODO: Set initial robot state and plan trajectories 
    sim_instance.step(1)

    # Reading the state 
    s, ds, tau = sim_instance.get_state()
    try:
        H_b = sim_instance.get_base()
    except RuntimeError as e:
        return sim_instance.get_simulation_time()
    w_b = sim_instance.get_base_velocity()
    t = sim_instance.get_simulation_time()

    # MPC
    mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
    mpc.initialize_centroidal_integrator(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
    mpc_output = mpc.plan_trajectory()

    # Set loop variables 
    TIME_TH = 20.0

    # Define number of steps
    n_step = int(
        TSID_controller_instance.frequency / sim_instance.get_simulation_frequency()
    )
    n_step_mpc_tsid = int(mpc.get_frequency_seconds() / TSID_controller_instance.frequency)

    counter = 0
    mpc_success = True
    energy_tot = 0.0
    succeded_controller = True

    # Simulation-control loop
    if simulator == "drake" and visualise_flag:
        sim_instance.meshcat.StartRecording()

    while t < TIME_TH:
        
        # Reading robot state from simulator
        s, ds, tau = sim_instance.get_state()
        energy_i = np.linalg.norm(tau)
        try:
            H_b = sim_instance.get_base()
        except RuntimeError as e:
            return sim_instance.get_simulation_time()

        w_b = sim_instance.get_base_velocity()
        t = sim_instance.get_simulation_time()

        # Update TSID
        TSID_controller_instance.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)

        # MPC plan 
        if counter == 0:
            mpc.set_state_with_base(s=s, s_dot=ds, H_b=H_b, w_b=w_b, t=t)
            mpc.update_references()
            mpc_success = mpc.plan_trajectory()
            mpc.contact_planner.advance_swing_foot_planner()
            if not (mpc_success):
                logging.error("MPC failed")
                break

        # Reading new references
        com, dcom, forces_left, forces_right = mpc.get_references()
        left_foot, right_foot = mpc.contact_planner.get_references_swing_foot_planner()

        # Update references TSID 
        TSID_controller_instance.update_task_references_mpc(
            com=com,
            dcom=dcom,
            ddcom=np.zeros(3),
            left_foot_desired=left_foot,
            right_foot_desired=right_foot,
            s_desired=np.array(s_des),
            wrenches_left=forces_left,
            wrenches_right=forces_right,
        )

        # Run control 
        succeded_controller = TSID_controller_instance.run()

        if not (succeded_controller):
            logging.error("Controller failed")
            return sim_instance.get_simulation_time()

        tau = TSID_controller_instance.get_torque()

        # Step the simulator
        sim_instance.set_input(tau)
        # sim_instance.step_with_motors(n_step=n_step, torque=tau)
        try:
            sim_instance.step(n_step=n_step)
        except RuntimeError as e:
            logging.error(e)
            return sim_instance.get_simulation_time()

        counter = counter + 1

        if counter == n_step_mpc_tsid:
            counter = 0
    
    if simulator == "drake" and visualise_flag:
        sim_instance.meshcat.StopRecording()
        sim_instance.meshcat.PublishRecording()
        # sim_instance.close_visualization()


    return sim_instance.get_simulation_time()

In [33]:
class GainTuningProblem:
    # @staticmethod
    def fitness(self, x):
        with redirect_stdout(open(os.devnull, "w")):
            result = -cost_function(x, simulator="mujoco") - cost_function(x, simulator="drake")
        return [result]

    def get_bounds(self):
        return ([0] * 28, [1000] * 28)

In [34]:
%%time

# [Warning] 
# This cell can take a long time to run.
# Choose the number of workers, population and generations based on the machine you are using.
num_workers = 2
population = 5
generations = 5

prob = pg.problem(GainTuningProblem())
algo = pg.algorithm(pg.cmaes(gen=generations, force_bounds=True))
algo.set_verbosity(1)
archi = pg.archipelago(
    n=num_workers,
    algo=algo,
    prob=prob,
    pop_size=population,
)
archi.evolve()
print(archi)
ret_flag = archi.wait_check()
res = archi.get_champions_f()
x = archi.get_champions_x()
print(archi, ret_flag)

dict_log = {}
dict_log["res"] = res
dict_log["x"] = x
dict_log["num_workers"] = num_workers
dict_log["population"] = population
dict_log["generations"] = generations
dict_log["ret_flag"] = ret_flag

print(dict_log)

[DEBUG] [2023-12-15 18:58:31.481] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'verbosity' not found.
[INFO] [2023-12-15 18:58:31.481] [thread: 27686] [blf] [QPTSID::initialize] 'verbosity' not found. The following parameter will be used 'false'.
[DEBUG] [2023-12-15 18:58:31.481] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 18:58:31.481] [thread: 27686] [blf] [CoMTask::initialize] [CoMTask Task.] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 18:58:31.481] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 18:58:31.481] [thread: 27686] [blf] [SE3Task::initialize]  [SE3Task Optimal Control Element - Frame name:  l_sole] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 18:58:31.481] [thread: 27686] [blf] [StdImplementation

[ERROR] [2023-12-15 18:58:31.862] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:31.871] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:31.882] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:32.024] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:32.033] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:32.180] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:32.189] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:32.203] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:32.217

[DEBUG] [2023-12-15 18:58:33.261] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'verbosity' not found.
[INFO] [2023-12-15 18:58:33.261] [thread: 27686] [blf] [QPTSID::initialize] 'verbosity' not found. The following parameter will be used 'false'.
[DEBUG] [2023-12-15 18:58:33.261] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 18:58:33.261] [thread: 27686] [blf] [CoMTask::initialize] [CoMTask Task.] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 18:58:33.261] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 18:58:33.261] [thread: 27686] [blf] [SE3Task::initialize]  [SE3Task Optimal Control Element - Frame name:  l_sole] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 18:58:33.261] [thread: 27686] [blf] [StdImplementation

[ERROR] [2023-12-15 18:58:33.618] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.636] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.650] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.662] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.674] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.693] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.707] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.848] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 18:58:33.869

[DEBUG] [2023-12-15 18:58:34.015] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'verbosity' not found.
[INFO] [2023-12-15 18:58:34.015] [thread: 27686] [blf] [QPTSID::initialize] 'verbosity' not found. The following parameter will be used 'false'.
[DEBUG] [2023-12-15 18:58:34.015] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 18:58:34.015] [thread: 27686] [blf] [CoMTask::initialize] [CoMTask Task.] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 18:58:34.016] [thread: 27686] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 18:58:34.016] [thread: 27686] [blf] [SE3Task::initialize]  [SE3Task Optimal Control Element - Frame name:  l_sole] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 18:58:34.016] [thread: 27686] [blf] [StdImplementation

...

[ERROR] [2023-12-15 19:01:23.794] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.


[DEBUG] [2023-12-15 19:01:24.146] [thread: 28100] [blf] [QPTSID::advance] The solver found an inaccurate feasible solution.


[ERROR] [2023-12-15 19:01:24.155] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:01:24.164] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:01:24.317] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:01:24.326] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:01:24.335] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
ERROR:root:Controller failed


[DEBUG] [2023-12-15 19:01:24.455] [thread: 28100] [blf] [StdImplementation::getParameterPrivate] Parameter named 'verbosity' not found.
[INFO] [2023-12-15 19:01:24.455] [thread: 28100] [blf] [QPTSID::initialize] 'verbosity' not found. The following parameter will be used 'false'.
[DEBUG] [2023-12-15 19:01:24.455] [thread: 28100] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 19:01:24.455] [thread: 28100] [blf] [CoMTask::initialize] [CoMTask Task.] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 19:01:24.456] [thread: 28100] [blf] [StdImplementation::getParameterPrivate] Parameter named 'mask' not found.
[INFO] [2023-12-15 19:01:24.456] [thread: 28100] [blf] [SE3Task::initialize]  [SE3Task Optimal Control Element - Frame name:  l_sole] Unable to find the mask parameter. The default value is used: true true true.
[DEBUG] [2023-12-15 19:01:24.456] [thread: 28100] [blf] [StdImplementation

[ERROR] [2023-12-15 19:01:26.515] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:01:26.529] [thread: 28100] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
Function solver (0x5596435e5290)
Input 0 (x0): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [36]:
%%time
# extract the best solution
best_x_k = np.array(x[np.argmin(res)])

# or load a known solution
# best_x_k = np.array([684.25282492, 648.05228734, 569.96763936, 350.65899653,
#        949.98434623, 515.09198331, 845.83335962,  17.95951342,
#        314.5705753 , 676.403354  , 440.72680377, 467.56525625,
#        681.69608946,  97.4551466 , 715.55867788, 416.80935819,
#        470.72979393, 680.49815841, 876.67111456, 654.65448001,
#        329.59585922,  65.34583639, 975.68181074, 342.1368993 ,
#        237.81553041, 606.40982421,  32.24741521,  10.83276846])

# or use default values
best_x_k = None

# run the cost function
cost_function(best_x_k, simulator="mujoco", visualise_flag = True)

INFO:drake:Meshcat listening for connections at http://localhost:7002


This is Ipopt version 3.14.12, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:      126
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      142

Total number of variables............................:       27
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       21
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  5.2479115e-03 1.00e+00 1.95e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

[ERROR] [2023-12-15 19:03:04.260] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:03:04.272] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:03:04.284] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
[ERROR] [2023-12-15 19:03:04.293] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.
ERROR:root:Controller failed
[ERROR] [2023-12-15 19:03:04.303] [thread: 27686] [blf] [QPTSID::advance] osqp was not able to find a feasible solution.


[DEBUG] [2023-12-15 19:03:04.249] [thread: 27686] [blf] [QPTSID::advance] The solver found an inaccurate feasible solution.
CPU times: user 1min 22s, sys: 3min 3s, total: 4min 25s
Wall time: 20.1 s


12.080999999999786